In [1]:
import os
os.environ['HF_HOME'] = '/run/cache/'


In [2]:
import pandas as pd
from tqdm.auto import tqdm
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import login

login()  

In [3]:
os.getenv('HF_HOME')

'/run/cache/'

In [5]:
torch.random.manual_seed(0)


In [6]:
with open('diabetes_data_with_ids', 'r') as file:
    diabetes_data_with_ids = json.load(file)

dict

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct", 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

config.json:   0%|          | 0.00/3.48k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


...

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [8]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         100G   50G   51G  50% /
tmpfs            64M     0   64M   0% /dev
tmpfs           7.7G     0  7.7G   0% /sys/fs/cgroup
/dev/nvme0n1p1  100G   50G   51G  50% /run
tmpfs            14G   36K   14G   1% /dev/shm
/dev/nvme2n1    2.0G  148M  1.8G   8% /home/jovyan
tmpfs            14G  120K   14G   1% /home/jovyan/.saturn
tmpfs            14G   12K   14G   1% /run/secrets/kubernetes.io/serviceaccount
tmpfs           7.7G   12K  7.7G   1% /proc/driver/nvidia
tmpfs           7.7G  5.8M  7.7G   1% /run/nvidia-persistenced/socket
tmpfs           7.7G     0  7.7G   0% /proc/acpi
tmpfs           7.7G     0  7.7G   0% /sys/firmware


In [9]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)


generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.9,  
    "top_k": 50,         
    "do_sample": True,  
}

In [23]:
torch.cuda.empty_cache()

In [42]:
prompt_template = '''
You are emulating a diabetes patient.
Formulate just 2 insightful questions based on the following diabetes conversation record. 
Use the context provided to generate relevant questions, but avoid directly copying words from the response.
Be very brief, as brief as possible
Note: Questions should not be more than 15 words max and all generated response should not be more than 40 words
Dont't repeat question
Record:
Context: {question}
return reult in this format
[question1,question2]

Dont include the instruction or anyother rubbish in the response i need only the generated response


'''

In [43]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)
    
    result = pipe(prompt, **generation_args)

    json_response = result
    return json_response

In [44]:
import random

sampledata = random.sample(diabetes_data_with_ids, 3)


In [45]:
sampledata[1]

{'question': 'What specific method does the booklet propose for informing family and friends about diabetes?',
 'answer': 'Answer: The booklet recommends sharing the booklet itself with family and friends as a means of educating them about diabetes and encouraging their support.',
 'id': '173ac5fc'}

In [46]:
results = {}
for doc in tqdm(sampledata): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions
    if doc_id == 'bedc96fa':
        print(questions)

  0%|          | 0/3 [00:00<?, ?it/s]

In [49]:
sampledata

[{'question': 'Why is it recommended for diabetic individuals to have regular dilated eye exams?',
  'answer': 'Regular dilated eye exams are essential for diabetic patients because diabetes increases their risk of developing diabetic retinopathy, a condition that can damage the blood vessels in the eyes and lead to vision loss. Dilated eye exams enable doctors to detect early signs of this condition and provide appropriate treatment.',
  'id': 'f97789ef'},
 {'question': 'What specific method does the booklet propose for informing family and friends about diabetes?',
  'answer': 'Answer: The booklet recommends sharing the booklet itself with family and friends as a means of educating them about diabetes and encouraging their support.',
  'id': '173ac5fc'},
 {'question': 'Besides managing blood sugar levels, what other health-related goals should individuals with diabetes strive for?',
  'answer': 'Answer:  Along with blood sugar control, individuals with diabetes should aim for a healt

In [60]:
results['7f51bcb9'][0]['generated_text'].strip()

"Question 1: What lifestyle changes can enhance the management of diabetes?\nQuestion 2: Are there specific dietary recommendations for diabetics to control blood sugar more effectively?\n** Answer: [Adhering strictly to the context, avoiding direct repetition, and maintaining brevity.] \nQuestion 1: How do routine medical check-ups benefit diabetics?\nQuestion 2: In what ways does mental health affect diabetes management?\n** Answer: [Tailored to the context with brevity and direct relevance.]\nQuestion 1: What role does exercise play in diabetes control?\nQuestion 2: How important is stress management for individuals with diabetes?\n** Answer: [Questions directly engaging with the context, maintaining brevity.]\nQuestion 1: What impact does diabetes have on cardiovascular health?\nQuestion 2: How can diabetics prevent common complications associated with the condition?\n** Answer: [Questions focused on the context, brief and to the point.]\nQuestion 1: What are the benefits of mainta